In [ ]:
import pandas as pd
import math
import operator

In [ ]:
df_stocks = pd.read_csv('Data\Brands_Categories_Sold_Lhr_encoded.csv')
df_stocks['dates'] = pd.to_datetime(df_stocks['dates'], dayfirst = True)
df_stocks.rename(columns={'dates':'Date'}, inplace=True)
df_stocks.set_index('Date', inplace=True)

df_stocks_features = pd.read_csv('Data\stocks_features.csv')
df_stocks_features['dates'] = pd.to_datetime(df_stocks_features['dates'], dayfirst = True)
df_stocks_features.rename(columns={'dates':'Date'}, inplace=True)
df_stocks_features.set_index('Date', inplace=True)

df_ItemList = pd.read_csv('Data\ItemList.csv')
df_Sales = pd.read_csv('Data\Lahore_sales_report.csv')


In [ ]:
df_stocks_monthly = df_stocks.resample('M').sum()

In [ ]:
# df_stocks_monthly
# df_stocks_features

In [ ]:
# bringing top 20% categories: top 20% by rank


category_monthly_sold = {}
brand_monthly_sold = {}

category_names = []
brand_names = []

for column_name, content in df_stocks_monthly.iteritems():
    if 'category' in column_name:
        if column_name not in category_names:
            category_names.append(column_name)
        if column_name not in category_monthly_sold:
            category_monthly_sold[column_name] = []

            for index, value in content.iteritems():
                category_monthly_sold[column_name].append((index.month_name() + str(index.year), value))      
    elif 'brand' in column_name:
        if column_name not in brand_names:
            brand_names.append(column_name)
        if column_name not in brand_monthly_sold:
            brand_monthly_sold[column_name] = []

            for index, value in content.iteritems():
                brand_monthly_sold[column_name].append((index.month_name() + str(index.year), value))      
                
    

In [ ]:
monthly_category_sold = {}
monthly_brand_sold = {}

for column_name, content in df_stocks_monthly.iteritems():
    if 'category' in column_name:
        for index, value in content.iteritems():
            month_year = index.month_name() + str(index.year)
            if month_year not in monthly_category_sold:
                monthly_category_sold[month_year] = []
                monthly_category_sold[month_year].append((column_name, value))
            else:
                monthly_category_sold[month_year].append((column_name, value))
    elif 'brand' in column_name:
        for index, value in content.iteritems():
            month_year = index.month_name() + str(index.year)
            if month_year not in monthly_brand_sold:
                monthly_brand_sold[month_year] = []
                monthly_brand_sold[month_year].append((column_name, value))
            else:
                monthly_brand_sold[month_year].append((column_name, value))




In [ ]:
monthly_category_sold

In [ ]:
top_monthly_category_sold = {}
for key in monthly_category_sold:
    sorted_cat = sorted(monthly_category_sold[key], key=lambda x:(x[1], x[1]), reverse=True)
    total_units_sold = sum(i for _, i in sorted_cat)
    main_categories = []
    units_sold = 0
    for c in sorted_cat:
        if units_sold >= 0.95*(total_units_sold):
            break
        else:
            units_sold += c[1]
            main_categories.append(c[0])
    top_monthly_category_sold[key] = main_categories


In [ ]:
top_monthly_category_sold

In [ ]:
top_20Perc_monthly_category_sold = {}
for key in monthly_category_sold:
    sorted_cat = sorted(monthly_category_sold[key], key=lambda x:(x[1], x[1]), reverse=True)
    top_20Perc_monthly_category_sold[key] = sorted_cat[:math.ceil( len(sorted_cat)*0.2 )]

top_20Perc_monthly_brand_sold = {}
for key in monthly_brand_sold:
    sorted_brand = sorted(monthly_brand_sold[key], key=lambda x:(x[1], x[1]), reverse=True)
    top_20Perc_monthly_brand_sold[key] = sorted_brand[:math.ceil( len(sorted_brand)*0.2 )]

In [ ]:
cat_freq = {}
for key in top_monthly_category_sold:
    for value in top_monthly_category_sold[key]:
        if value not in cat_freq:
            cat_freq[value] = 1
        else:
            cat_freq[value]+=1

In [ ]:
print(dict( sorted(cat_freq.items(), key=operator.itemgetter(1),reverse=True) ))

In [ ]:
top_95_percent_overall_categories = []
for key in cat_freq:
    if cat_freq[key] > 12:
        top_95_percent_overall_categories.append(key)

In [ ]:
b_c_encoded = {}

In [ ]:
top_95_percent_overall_categories_decoded = []
for c in top_95_percent_overall_categories:
    for key in b_c_encoded:
        if b_c_encoded[key] == c:
            top_95_percent_overall_categories_decoded.append(key)

In [ ]:
top_95_percent_overall_categories_decoded

In [ ]:
category_wise_brands = {}
for c in top_95_percent_overall_categories_decoded:
    category_wise_brands[c] = []
    for index, row in df_ItemList.iterrows():
        if row['Category'] == c:
            if row['ItemName'] not in category_wise_brands[c]:
                category_wise_brands[c].append(row['ItemName'])
        

In [ ]:
category_dfs = {}
for key in category_wise_brands:
    category_dfs[key] = pd.DataFrame(columns=['Date'] + category_wise_brands[key])


In [ ]:
category_dfs

In [ ]:
for key in category_dfs:
    print(key)
    initial_row = [ 0 for i in range( len(category_dfs[key].columns)) ]

    for index_sales, row_sales in df_Sales.iterrows():

        date = row_sales['Customer Date']
        itemcode = row_sales['Design Number']
        qty_sold = row_sales['Quantity']

        for index_il, row_il in df_ItemList.iterrows():
            
            if itemcode == row_il['ItemCode'] and key == row_il['Category']:
                
                brand = row_il['ItemName']
                df_length = len(category_dfs[key])
                category_dfs[key].loc[df_length] = initial_row
                category_dfs[key].loc[df_length, 'Date'] = date
                category_dfs[key].loc[df_length, brand] = qty_sold
                
            

In [ ]:
for key in category_dfs:
    filename = key.replace("/", "")
    category_dfs[key].to_csv(filename + '.csv')

In [ ]:
df_ItemList = pd.read_csv('Data\ItemList.csv')

In [ ]:
cat_freq_noncoded = {}
for key in cat_freq:
    for c in b_c_encoded:
        if b_c_encoded[c] == key:
            cat_freq_noncoded[c] = cat_freq[key] 
    

In [ ]:
# cat_freq_noncoded
# print(dict( sorted(cat_freq_noncoded.items(), key=operator.itemgetter(1),reverse=True) ))

In [ ]:
cat_freq = dict( sorted(cat_freq.items(), key=operator.itemgetter(1),reverse=True) )

In [ ]:
cat_freq

In [ ]:
# even if its sold once a year then thats fine

In [ ]:
main_categories = {}
for key in cat_freq:
    if cat_freq[key] >= math.floor(len(cat_freq)*0.25):
        main_categories[key] = cat_freq[key]

In [ ]:
main_categories
main_categories_names = []
for key in main_categories:
    main_categories_names.append(key)
main_categories_names

In [ ]:
df_main_cat = df_stocks[main_categories]

In [ ]:
# df_main_cat = df_main_cat.loc[:, ~df_main_cat.columns.isin(brand_names)]

In [ ]:
df_main_cat

In [ ]:
columns_needed = ['Average Discount', 'Year', 'Month', 'Day', 'Week Day', 'Week End', 'Hijri Dates', 'eid ul fitr', 'eid ul Adha', 'Muharrum', 'Wheat Harvest', 'Cotton Harvest'] + main_categories_names

In [ ]:
columns_needed

In [ ]:
df_main_cat_features = df_stocks_features[columns_needed]

In [ ]:
df_main_cat_features

In [ ]:
df_main_cat_features.to_csv('Data\main_categories_features.csv')